In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


from ema_workbench import load_results
from ema_workbench.analysis import prim
from custom_problem_formulation_no_RfR import get_model_for_problem_formulation

In [6]:
# First we test the final policy set, against each scenario, which remained after the PRIM analysis.

model, steps = get_model_for_problem_formulation()

final_policies = pd.read_csv('results/levers_final_policies.csv')
# final_policies = final_policies.rename(columns={'Unnamed: 0': 'policy'})
final_policies = final_policies[[l.name for l in model.levers]]

policies = []
for idx, row in final_policies.iterrows():
    policy_dict = row.to_dict()
    policy_name = idx
    policies.append(Policy(policy_name, **policy_dict))

combined_df = pd.read_csv('results/combined_df.csv')    # NOG genereren
scenarios_df = combined_d[[u.name for u in model.uncertainties]]

scenarios = []
for idx, row in scenarios_df.iterrows():
    scenario_dict = row.to_dict()
    scenario_name = idx
    scenarios.append(Scenario(scenario_name, **scenario_dict))


,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
0,0,0,0,0,0,1,1,1,1,1,...,0,0,7,6,6,6,6,2,1,7
1,1,1,0,1,0,0,1,1,1,1,...,0,6,7,2,7,7,6,4,4,7
2,1,0,0,1,1,0,1,1,1,1,...,0,3,1,1,6,3,4,6,3,1
3,1,1,0,0,0,0,1,1,1,1,...,0,5,4,6,0,4,2,7,6,4


In [ ]:
  elif args.mode == 'vulnerability':

        dike_model, planning_steps = get_model_for_problem_formulation('All Dikes')

        # ~3-7 policies from final Robustness analysis
        policies_df = pd.read_csv('./output/policies__final_set.csv',
                                index_col='Policy Name')
        policies_df = policies_df[[L.name for L in dike_model.levers]]

        policies = []
        for name, policy in policies_df.iterrows():
            policies.append(Policy(str(name), **policy.to_dict()))

        # PRIM-box scenarios from original DFE
        scenarios_fn = 'base_case_results__100000_scenarios__prim_filtered.csv'
        scenarios_df = pd.read_csv(f'./output/{scenarios_fn}')
        scenarios_df = scenarios_df[[U.name for U in dike_model.uncertainties]]

        scenarios = []
        for name, scenario in scenarios_df.iterrows():
            scenarios.append(Scenario(str(name), **scenario.to_dict()))